In [ ]:
import pandas as pd
import datetime as dt

# Load JSON data
df = pd.read_json("C:/Users/Dileep Sathya/OneDrive/Desktop/Stock_AI_2.0/artifacts/hist_data.json")

# Ensure 'date' is datetime
df['date'] = pd.to_datetime(df['date'])

# Add year column
df['year'] = df['date'].dt.year

# Create previous day features
df['prev_day_open'] = df.groupby('symbol')['open'].shift(1)
df['prev_day_high'] = df.groupby('symbol')['high'].shift(1)
df['prev_day_low'] = df.groupby('symbol')['low'].shift(1)
df['prev_day_close'] = df.groupby('symbol')['close'].shift(1)

df = df.dropna()

# Calculate previous day features
df['prev_day_range'] = abs((df['prev_day_high'] - df['prev_day_low']) / df['prev_day_low']) * 100
df['prev_day_body_range'] = abs((df['prev_day_close'] - df['prev_day_open']) / df['prev_day_open']) * 100

df = df.dropna()

# Filter for specific symbol
include_list2 = ['ONGC']
df = df[df['symbol'].isin(include_list2)]


# Apply eligibility condition
df['eligibility'] = (df['prev_day_range'] < 1.75) & (df['prev_day_body_range'] < 0.5)

# Current day ranges
df['range'] = abs((df['high'] - df['low']) / df['low']) * 100
df['body_range'] = abs((df['close'] - df['open']) / df['open']) * 100

# Drop unused columns
df = df.drop(columns=[
    'open', 'high', 'low', 'close', 'volume', 'year',
    'prev_day_high', 'prev_day_low', 'prev_day_close', 'prev_day_open'
])

# Filter for past 366 days only
today = pd.to_datetime("2025-06-16")
start_date = today - pd.Timedelta(days=183)
df = df[(df['date'] >= start_date) & (df['date'] <= today)]
df = df.sort_values('date')

# Final filtered DataFrame based on eligibility
fil_df = df[df['eligibility'] == True]


In [12]:
fil_df

,date,symbol,prev_day_range,prev_day_body_range,eligibility,range,body_range
79391,2024-12-27,ONGC,1.255230,0.104167,True,1.988997,1.291667
79395,2025-01-02,ONGC,1.222763,0.361633,True,4.126582,3.805104
79411,2025-01-24,ONGC,1.018494,0.019011,True,3.867058,2.463972
79449,2025-03-20,ONGC,1.528497,0.030108,True,2.168010,0.903718
79452,2025-03-25,ONGC,1.503605,0.147426,True,2.003741,0.721282
79458,2025-04-03,ONGC,1.411290,0.235953,True,2.931838,1.891129
79463,2025-04-11,ONGC,1.725234,0.027027,True,2.730300,0.906702
79469,2025-04-23,ONGC,1.350805,0.477969,True,3.196962,1.378950
79471,2025-04-25,ONGC,1.358871,0.252000,True,2.569558,1.072289
79486,2025-05-19,ONGC,1.295695,0.085000,True,1.853742,0.572581


In [13]:
df['prev_day_body_range'].describe()

count    123.000000
mean       1.393091
std        1.099625
min        0.019011
25%        0.671495
50%        1.121272
75%        1.869067
max        6.048387
Name: prev_day_body_range, dtype: float64

In [14]:
df['prev_day_range'].describe()

count    123.000000
mean       2.706347
std        1.240749
min        0.947674
25%        1.870834
50%        2.394749
75%        3.246898
max        7.707317
Name: prev_day_range, dtype: float64